In [1]:
from Bio import Entrez, SeqIO
from Bio.Seq import Seq

from Bio.SeqRecord import SeqRecord
from Bio import Entrez

import pysam
import vcf
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import functools
import re

import numpy as np
import pandas as pd
import math

Entrez.email = "simon.burgermeister@gmail.com"



## Create SAM (sequence alignement map):
<br>
Store sequences alifgnement to ref sequence:
<br>
Use bowtie2, install with: conda install -c bioconda bowtie2
<br>
 
###  Create ref se (index)

<br>
We use NC_045512.2
<br>

In [8]:
!bwa index sequences3/ref/ref_sequence.fasta

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index sequences3/ref/ref_sequence.fasta
[main] Real time: 0.016 sec; CPU: 0.010 sec


In [10]:
#Align to ref:
#!bowtie2 -x test_seq/ref_seq/ref -f sequences2/sequences.fasta -S sequences2/aligned.sam
!bwa mem sequences3/ref/ref_sequence.fasta sequences3/all.fasta > sequences3/aligned.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 11 sequences (327935 bp)...
[M::mem_process_seqs] Processed 11 reads in 0.272 CPU sec, 0.272 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem sequences3/ref/ref_sequence.fasta sequences3/all.fasta
[main] Real time: 0.274 sec; CPU: 0.273 sec


### Create BAM file and VCF:
<br>
When using fasta we do not utilize the quality score.
<br>
<br>

In [11]:
!samtools view -S -b sequences3/aligned.sam > sequences3/aligned.bam
!samtools sort sequences3/aligned.bam -o sequences3/sorted.bam

In [12]:
!samtools index sequences3/sorted.bam

In [13]:
!samtools mpileup -uf sequences3/ref/ref_sequence.fasta sequences3/sorted.bam | bcftools call -mv  --multiallelic-caller --variants-only  > sequences3/var.raw.vcf

Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


## Check VCF:

<br>
<br>

In [14]:
import vcf
from collections import defaultdict

f = vcf.Reader(filename='sequences3/var.raw.vcf')

my_type = defaultdict(int)
num_alts = defaultdict(int)

for rec in f:
    my_type[rec.var_type, rec.var_subtype] += 1
    print(rec.var_type)
    if rec.is_snp:
        num_alts[len(rec.ALT)] += 1
    else:
        print(rec)
print(my_type)
print(num_alts)

snp
snp
snp
snp
snp
snp
snp
snp
snp
snp
indel
Record(CHROM=NC_045512.2, POS=11287, REF=GTCTGGTTTT, ALT=[G])
snp
snp
snp
snp
snp
snp
indel
Record(CHROM=NC_045512.2, POS=21764, REF=ATACATGT, ALT=[AT])
indel
Record(CHROM=NC_045512.2, POS=21990, REF=TTTATTA, ALT=[TTTA])
snp
snp
snp
snp
snp
snp
snp
snp
snp
snp
snp
snp
snp
indel
Record(CHROM=NC_045512.2, POS=28270, REF=TAAAA, ALT=[TAAA])
snp
snp
snp
snp
snp
snp
snp
defaultdict(<class 'int'>, {('snp', 'ts'): 21, ('snp', 'tv'): 15, ('indel', 'del'): 4})
defaultdict(<class 'int'>, {1: 36})
